# Recommendations
The following code reads in a list of purchase events and generates a simple 

In [34]:
path = "data.txt"
session = SparkSession.builder \
  .appName("spark session example") \
  .getOrCreate()

df = session.read.option("header","true").option("inferSchema", "true").option("delimiter", '|').csv(path)
df.createOrReplaceTempView("events")

users_products = df.select("product_id", "user_id").distinct()
users_products.createOrReplaceTempView("users_products")


In [111]:
recommendations = sqlContext.sql("SELECT users_products.product_id AS parent_product_id, \
                                  events.product_id AS related_product_id, \
                                  COUNT(*) AS count \
                                  FROM users_products INNER JOIN events on events.user_id = users_products.user_id \
                                  GROUP BY users_products.product_id,  events.product_id \
                                  HAVING COUNT(*) > 15")

In [112]:
recommendations.show()

+-----------------+------------------+-----+
|parent_product_id|related_product_id|count|
+-----------------+------------------+-----+
|                3|                 1|   20|
|                2|                 2|   18|
|                8|                 9|   16|
|                8|                 3|   16|
|                5|                 6|   18|
|                1|                 1|   28|
|                1|                 3|   16|
|                8|                10|   16|
|               10|                 1|   18|
|                3|                 3|   21|
|                8|                 1|   18|
|                8|                 8|   21|
|                8|                 2|   17|
|                9|                 5|   16|
|                5|                 8|   16|
|                5|                 1|   20|
|                9|                 9|   19|
|                7|                 7|   17|
|                4|                 4|   16|
|         

In [113]:
recommendations.select("parent_product_id", "related_product_id").rdd.groupByKey().map(lambda x: (x[0], list(x[1]))).collect()

[(1, [1, 3]),
 (2, [2]),
 (3, [1, 3]),
 (4, [4]),
 (5, [6, 8, 1, 5]),
 (6, [6]),
 (7, [7]),
 (8, [9, 3, 10, 1, 8, 2, 6]),
 (9, [5, 9, 6]),
 (10, [1, 10])]